In [ ]:
# # USING NLP WE ARE DETECTING THE CITY IN A GIVEN SENTENCE

In [1]:
import requests
import spacy
from fuzzywuzzy import fuzz
from opencage.geocoder import OpenCageGeocode
from langdetect import detect
from googletrans import Translator

C:\ProgramData\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 393.8 kB/s eta 0:00:33
     --------------------------------------- 0.1/12.8 MB 804.6 kB/s eta 0:00:16
      --------------------------------------- 0.2/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.5/12.8 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.9/12.8 MB 3.0 MB/s eta 0:00:04
     --- ------------------------------------ 1.2/12.8 MB 3.7 MB/s eta 0:00:04
     ------ --------------------------------- 2.1/12.8 MB 5.8 MB/s eta 0:00:02
     ---------- ----------------------------- 3.2/12.8 MB 7.6 MB/s eta 0:00:02
     -------------- ------------------------- 4.6/12.8 MB 9.9 MB/s eta 0:00:01
     ------------------ --------------------- 6.0/12.8 MB 11.9 MB/s eta 0:00:01
     --------------------- ------------------ 6.9/12.8 MB 13.0 

In [5]:
# Load spaCy's English language model
nlp = spacy.load('en_core_web_sm')

def extract_location_entities(sentence, lang='en'):
    if lang != 'en':
        translator = Translator()
        sentence = translator.translate(sentence, src=lang, dest='en').text

    doc = nlp(sentence)
    locations = []

    for ent in doc.ents:
        if ent.label_ == 'GPE':  # GPE: Geo-Political Entity (location)
            locations.append((ent.text, ent.start_char, ent.end_char))

    return locations

def get_location_info(location, api_key):
    try:
        geocoder = OpenCageGeocode(api_key)
        results = geocoder.geocode(location)

        if results and len(results):
            place = results[0]
            return {
                'name': place['formatted'],
                'country': place['components']['country'],
                'latitude': place['geometry']['lat'],
                'longitude': place['geometry']['lng']
            }
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

In [8]:

# Example usage
api_key = ""  # Replace with your actual OpenCageData API key
user_input = input("Enter the text (in any language) containing city or country names: ")

# Detect the language of the input text
lang = detect(user_input)

locations = extract_location_entities(user_input, lang)

if locations:
    print("Location Entities:")
    for location in locations:
        info = get_location_info(location[0], api_key)
        if info:
            print(f"Original Name: {location[0]}")
            print(f"English Name: {info['name']}")
            print(f"Country: {info['country']}")
            print(f"Latitude: {info['latitude']}, Longitude: {info['longitude']}")
            print("=" * 30)
        else:
            print(f"Location '{location[0]}' not found.")
else:
    print("No location entities found in the sentence.")




Enter the text (in any language) containing city or country names: القاهرة هي عاصمة مصر
Location Entities:
Original Name: Cairo
English Name: Cairo, Egypt
Country: Egypt
Latitude: 30.0443879, Longitude: 31.2357257
Original Name: Egypt
English Name: Egypt
Country: Egypt
Latitude: 26.2540493, Longitude: 29.2675469
